# Understanding the Amount of MIMIC Notes

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [4]:
cd .

/scratch/ssd001/home/vinithms/ML_Linkage_Attacks/notebooks


In [5]:
NOTES_CSV='/scratch/hdd001/home/vinithms/mimic_bert/NOTEEVENTS.csv'

In [6]:
notes_chunks_csv = pd.read_csv(NOTES_CSV,chunksize=100000)

In [7]:
notes_df = pd.concat(notes_chunks_csv)

/pkgs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
notes_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [24]:
notes_df.CATEGORY.value_counts()

Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: CATEGORY, dtype: int64

In [25]:
data_dir="/scratch/hdd001/home/vinithms/ml_linkage_attacks/mimic-years/"
OUTCOMES_DIR=os.path.join(data_dir, 'static_data.csv')
max_time=24
gap_time=12
idx = pd.IndexSlice

## Combine with labels

In [26]:
    y_df=pd.read_csv(OUTCOMES_DIR)
    y_df=y_df.set_index(['subject_id', 'hadm_id'])
    #rename the index to match other columns
#     y_df.index.rename([name.encode() for name in y_df.index.names], inplace=True)

    #rename the columns to match other column levels
    column_names=y_df.columns.tolist()

    for i in range(3):
        y_df=pd.concat([y_df], axis=1, keys=['OUTCOMES']).swaplevel(0, 1, 1).swaplevel(0, 1, 1)
    for level, item in enumerate(['itemid'.encode(),'label'.encode(),'LEVEL1'.encode(),'LEVEL2'.encode()]):
        y_df.columns=y_df.columns.rename(item, level=level)
        y_df.columns.set_levels(column_names,level=level,inplace=True)

    y_df.columns.set_labels([[i for i in range(len(y_df.columns.levels[0]))] for i in range(4)],level=[0,1,2,3],inplace=True)

    y_df.loc[:, (slice(None), slice(None), slice(None), 'los_icu')]*=24 #icu data from days to hours

    mask = y_df.loc[:,(slice(None), slice(None), slice(None), 'los_icu')].values>max_time+gap_time
    y_df=y_df.loc[idx[mask, :, :], :]

    outcomes_df=y_df.copy()


    categories=['asian', 'black', 'hispanic','white']

    name=y_df.loc[:, idx[:,:,:,'ethnicity']].values.ravel()
    for category in categories:
        name[[i for i, item in enumerate(name) if category in item.lower()]]=category
    name[[i for i, item in enumerate(name) if item not in categories]]='other'

    for category in categories+['other']:
        y_df.loc[:, idx[category, category, category,category]]=[1 if item in category else 0 for item in name]

    #same for gender
    name=y_df.loc[:, idx[:,:,:,'gender']].values.ravel()
    y_df.loc[:, idx['F','F','F','F']]=[1 if 'f' in item.lower() else 0 for item in name]
    y_df.loc[:, idx['M','M','M','M']]=[1 if 'm' in item.lower() else 0 for item in name]




    
    #apply common indices
    label_df=outcomes_df.copy()

    label_df.columns=label_df.columns.droplevel([0,1,2]) # only do this once

    label_df['los_3']=np.zeros((len(label_df),1)).ravel()
    label_df.loc[label_df['los_icu']>=3*24, 'los_3']=1

/pkgs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: .set_labels was deprecated in version 0.24.0. Use .set_codes instead.
  from ipykernel import kernelapp as app


In [27]:
label_df.head()

,b'LEVEL2',icustay_id,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,los_3
subject_id,hadm_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,...,2101-10-26 20:43:09,145.5504,EMERGENCY,MICU,0,0,0,1,0,1.0
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,...,2191-03-17 16:46:31,40.2840,EMERGENCY,MICU,0,0,0,1,0,0.0
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,...,2175-06-03 13:39:54,88.1496,ELECTIVE,SICU,0,0,0,1,0,1.0
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,...,2149-11-14 20:52:14,127.7544,EMERGENCY,MICU,1,1,1,1,0,1.0
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,...,2178-04-17 20:21:05,38.0256,EMERGENCY,SICU,0,0,0,1,0,0.0


In [29]:
label_df.reset_index(level=0, inplace=True)

In [30]:
del label_df.columns.name

In [32]:
notes_df.shape

(2083180, 11)

In [33]:
subset_label_df = label_df[['hadm_id','subject_id','icustay_id','gender','ethnicity','age','insurance','mort_icu','readmission_30','los_3']]

In [34]:
subset_label_df.head()

,hadm_id,subject_id,icustay_id,gender,ethnicity,age,insurance,mort_icu,readmission_30,los_3
0,145834,3,211552,M,WHITE,76.5268,Medicare,0,0,1.0
1,185777,4,294638,F,WHITE,47.8450,Private,0,0,0.0
2,107064,6,228232,F,WHITE,65.9407,Medicare,0,0,1.0
3,150750,9,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,1,0,1.0
4,194540,11,229441,F,WHITE,50.1483,Private,0,0,0.0


In [35]:
notes_labels_df = pd.merge(notes_df,subset_label_df, left_on=['SUBJECT_ID','HADM_ID'], right_on=['subject_id','hadm_id'])

In [37]:
notes_labels_df.shape

(560843, 21)

In [21]:
nurse_phys_df = notes_labels_df.loc[notes_labels_df['CATEGORY'].isin(['Physician ','Nursing'])]

In [22]:
nurse_phys_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,hadm_id,subject_id,icustay_id,gender,ethnicity,age,insurance,mort_icu,readmission_30,los_3
527,729404,82559,197325.0,2157-02-02,2157-02-02 00:59:00,2157-02-02 06:10:58,Nursing,Nursing Progress Note,16934.0,NaN,...,197325,82559,259299,F,WHITE,85.9256,Medicare,0,0,0.0
528,729762,82559,197325.0,2157-02-03,2157-02-03 11:10:00,2157-02-03 11:10:15,Nursing,Nursing Transfer Note,15526.0,NaN,...,197325,82559,259299,F,WHITE,85.9256,Medicare,0,0,0.0
529,729768,82559,197325.0,2157-02-03,2157-02-03 11:10:00,2157-02-03 11:56:06,Nursing,Nursing Transfer Note,15526.0,NaN,...,197325,82559,259299,F,WHITE,85.9256,Medicare,0,0,0.0
530,729741,82559,197325.0,2157-02-03,2157-02-03 10:29:00,2157-02-03 10:29:46,Physician,Physician Attending Progress Note,16383.0,NaN,...,197325,82559,259299,F,WHITE,85.9256,Medicare,0,0,0.0
531,729747,82559,197325.0,2157-02-03,2157-02-03 07:46:00,2157-02-03 10:42:09,Physician,Physician Resident Progress Note,18480.0,NaN,...,197325,82559,259299,F,WHITE,85.9256,Medicare,0,0,0.0


In [23]:
nurse_phys_df.shape

(136693, 21)

In [31]:
nurse_phys_df_train,nurse_phys_df_val =  train_test_split(nurse_phys_df_train_val, test_size=0.2, random_state=42)

## Create Pretraining Text Files

In [121]:
with open('./data/mimic_notes/pretraining/nurse_phys_notes.raw.txt', 'w') as f:
    f.write(nurse_phys_df['TEXT'].str.cat(sep='\n'))

## Create Mortality TSV Files for Finetuning

In [155]:
nurse_phys_df_to_save = nurse_phys_df[['SUBJECT_ID','HADM_ID','gender','ethnicity','insurance','TEXT','readmission_30']]

In [156]:
grouped_df = nurse_phys_df_to_save.astype(str).groupby(['SUBJECT_ID','HADM_ID','gender','ethnicity','insurance', 'readmission_30']).agg(' '.join).reset_index()
                                                        
                                                        

In [157]:
nurse_phys_df_train_dev, nurse_phys_df_test = train_test_split(grouped_df, test_size=0.2, random_state=42, stratify=grouped_df['readmission_30'])


In [158]:
nurse_phys_df_train, nurse_phys_df_dev = train_test_split(nurse_phys_df_train_dev, test_size=0.15, random_state=42, stratify=nurse_phys_df_train_dev['readmission_30'])

In [159]:
nurse_phys_df_train.to_csv('./data/mimic_notes/readmission_30/nurse_phys_notes_train.tsv', sep='\t', index=False)

In [160]:
nurse_phys_df_dev.to_csv('./data/mimic_notes/readmission_30/nurse_phys_notes_dev.tsv', sep='\t', index=False)

In [161]:
nurse_phys_df_test.to_csv('./data/mimic_notes/readmission_30/nurse_phys_notes_test.tsv', sep='\t', index=False)

## Create LOS > 3 TSV Files for Finetuning

## Create Readmission in 30 TSV Files for Finetuning

In [36]:
with open('./data/nurse_phys_notes_train.raw.txt', 'w') as f:
    f.write(nurse_phys_df_train['TEXT'].str.cat(sep='\n'))

In [37]:
with open('./data/nurse_phys_notes_val.raw.txt', 'w') as f:
    f.write(nurse_phys_df_val['TEXT'].str.cat(sep='\n'))

In [38]:
with open('./data/nurse_phys_notes_test.raw.txt', 'w') as f:
    f.write(nurse_phys_df_test['TEXT'].str.cat(sep='\n'))